# Demo de Deep Autoencoder (DAE) para procesar los datos de tipos de Iris
Basado en: 

https://blog.keras.io/building-autoencoders-in-keras.html


https://towardsdatascience.com/deep-autoencoders-using-tensorflow-c68f075fd1a3

1) Importar librerías:

In [ ]:
import keras
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model
from keras.utils import plot_model

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os


print("\nLibrerías importadas")

2) Configurar los datos cargados (se debe definir de acuerdo a los datos):


In [ ]:
# define atributos y clases
CSV_COLUMN_NAMES = ['LargoSepalo', 'AnchoSepalo', 'LargoPetalo', 'AnchoPetalo', 'Clase']
CLASSES = ['Setosa', 'Versicolor', 'Virginica']

ClassAttributeName = 'Clase'

print("Configuración definida.")

3) Cargar CSV con datos a procesar y preparar datos para entrenar y probar (ya separados):

In [ ]:
# levanta los datos de entrenamiento y prueba
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

# ver datos
train.head()
#test.head()

In [ ]:
# define datos de entrada y salida para entrenamiento
x_train = np.array(train.drop([ClassAttributeName], axis=1))
y_train = np.array(train[ClassAttributeName])

# define datos de entrada y salida para testing
x_test = np.array(test.drop([ClassAttributeName], axis=1))
y_test = np.array(test[ClassAttributeName])

print("\n\nDatos Originales ", len(x_train)+len(x_test))
print("- Datos para Entrenar ", len(x_train))
print("- Datos para Probar ", len(x_test))

4) Definir la configuración del modelo DAE:

In [ ]:
# define tamaño de datos de entrada y salida
num_inputs = len(x_train[0])
num_outputs = num_inputs

# cantidad de neuronas ocultas para features (datos comprimidos o codings)
num_features = 2

# cantidad de neuronas ocultas para la parte Encoder 
#   (cada elemento de la lista es la cantidad de pesos que tiene cada una)
dae_layers = [ 3 ] 

#  agrega la capa de features a las capas
dae_layers.append( num_features ) 

# cantidad de neuronas ocultas para la parte Decoder 
#   (usa la la lista de Encoder inversa)
for eachEncLayer in dae_layers[0:len(dae_layers)-1][::-1]:
  dae_layers.append( eachEncLayer )

# cantidad de épocas del entrenamiento
# (a medida que la cantidad de capas ocultas se mayor y el nro de features es menor, 
#   se recomienda entrenar más épocas)
cantEpocas = 500

print("Configuración del DAE definida: [", num_inputs, dae_layers, num_outputs, "] ")

5) Creación del modelo DAE:

In [ ]:
# define la arquitectura de capas del Deep Autoencoder
# teniendo en cuenta la definición dada anteriomente
input_data_Lay = Input(shape=(num_inputs,), name='input_data') # capa de entrada
eachLay = input_data_Lay
auxName = 'enc_'
auxId = 1 
for num_hid in dae_layers:  

    # define el nombre de la capa oculta
    if num_features==num_hid:
        auxlayerName = 'features'
        auxName = 'dec_'
        auxId = auxId - 1
    else:
        auxlayerName = auxName+str(auxId)
        if auxName == 'enc_':
          auxId = auxId + 1
        else:
          auxId = auxId - 1

    # agrega la capa oculta
    eachLay = Dense(num_hid, activation='relu', name=auxlayerName)(eachLay) # capas ocultas

    if  auxlayerName == 'features':
      features_Lay = eachLay

output_data_Lay = Dense(num_outputs, activation=None, name='output_data')(eachLay) # capa de salida

# genera el modelo Deep Autoencoder
DAEmodel = Model(input_data_Lay, output_data_Lay, name='DAE')
#DAEmodel.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])
DAEmodel.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

print("Modelo DAE creado con ", len(DAEmodel.layers), " capas:")
DAEmodel.summary()
print("\n")
plot_model(DAEmodel, show_layer_names=True, show_shapes=True)

5) Entrenar el modelo DAE:

In [ ]:
# lleva a cabo el entrenamiento
# usando los mismos datos como entrada y salida
x_train_prep  = np.array(x_train).reshape(len(x_train), num_inputs)
DAEmodel.fit(x_train_prep, x_train_prep,
                epochs = cantEpocas, 
                batch_size = 15)


7) Evaluar el modelo DAE entrenado solicitando que reconstruya los datos de entrenamiento:

In [ ]:
# evalua al modelo 
resEval = DAEmodel.evaluate(x_train_prep, x_train_prep)
print("\n>Evaluación del Modelo: ")
print("    - Error: ", resEval[0])
print("    - Exactitud: ", resEval[1]*100)
print("\n")

# procesa las imágenes con el modelo 
reconstr_data = DAEmodel.predict(x_train_prep)

# muestra las 15 primeras imágenes 
print(">Ejemplos de Resultados: ")
for i in range(len(x_train))[:5]:
      
    # muestra la real
    print("  + Original: ", x_train[i])
    # muestra la generada por el modelo
    print("  * Reconstr: ", reconstr_data[i])
    print("  - Dif:      ", reconstr_data[i]-x_train[i])
    print("\n")  

print("> Total de Diferencias ", sum(reconstr_data-x_train))
print("> Promedio de Diferencias ", sum(reconstr_data-x_train)/len(x_train))

9) Probar el modelo DAE entrenado con los datos de test:

In [ ]:
x_test_prep  = np.array(x_test).reshape(len(x_test), num_inputs)

# evalua al modelo 
resEval_test = DAEmodel.evaluate(x_test_prep, x_test_prep)
print("\n>Evaluación del Modelo: ")
print("    - Error: ", resEval_test[0])
print("    - Exactitud: ", resEval_test[1]*100)
print("\n")

# procesa las imágenes con el modelo 
reconstr_data_test = DAEmodel.predict(x_test_prep)

# muestra las 15 primeras imágenes 
print(">Ejemplos de Resultados: ")
for i in range(len(x_test))[:5]:
      
    # muestra la real
    print("  + Original: ", x_test[i])
    # muestra la generada por el modelo
    print("  * Reconstr: ", reconstr_data_test[i])
    print("  - Dif:      ", reconstr_data_test[i]-x_test[i])
    print("\n")  

print("> Total de Diferencias ", sum(reconstr_data_test-x_test))
print("> Promedio de Diferencias ", sum(reconstr_data_test-x_test)/len(x_test))

8) A partir del modelo DAE entrenado, generar dos sub-modelos Encoder y Decoder:


*   Generar y usar el modelo Encoder para 'clusterizar' los datos de entrenamiento:


In [ ]:
## Generar el sub-modelo Encoder para Clustering
## (desde input hasta features)

# reutiliza las capas entrenadas del modelo DAE original
clust_input_Lay = input_data_Lay  # capa de entrada
clust_output_Lay =  features_Lay  # capa de salida

# genera el modelo
CLUSTmodel = Model(input_data_Lay, features_Lay, name='Encoder/Clustering')

print("> Modelo Encoder: ")
CLUSTmodel.summary()
plot_model(CLUSTmodel, show_layer_names=True, show_shapes=True)

In [ ]:
 # función auxiliar para generar un gráfico 
 def genera_grafico(datos, clases, titulo):  
    principalDf = pd.DataFrame(data = datos,
                columns = ['a', 'b'])
    finalDf = pd.concat([principalDf, 
                        pd.DataFrame(clases, columns = ['target'])], 
                        axis = 1)

    fig = plt.figure(figsize = (10,10))
    ax = fig.add_subplot(1,1,1) 
    ax.set_title(titulo, fontsize = 20)
    for target in set(clases):
        indicesToKeep = finalDf['target'] == target
        ax.scatter(finalDf.loc[indicesToKeep, 'a'],
                  finalDf.loc[indicesToKeep, 'b'],
                  s = 50)
    ax.legend(CLASSES)
    ax.grid()


# función auxiliar para generar un gráfico 
# con los valores codificados 
# usando PCA para simplificarlos en 2 ejes
def genera_grafico_pca(datos, clases, titulo):
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(datos)
    principalDf = pd.DataFrame(data = principalComponents,
                columns = ['pca_1', 'pca_2'])
    finalDf = pd.concat([principalDf, 
                        pd.DataFrame(clases, columns = ['target'])], 
                        axis = 1)

    fig = plt.figure(figsize = (10,10))
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel('PCA1', fontsize = 15)
    ax.set_ylabel('PCA2', fontsize = 15)
    ax.set_title(titulo, fontsize = 20)
    for target in set(clases):
        indicesToKeep = finalDf['target'] == target
        ax.scatter(finalDf.loc[indicesToKeep, 'pca_2'],
                  finalDf.loc[indicesToKeep, 'pca_1'],
                  s = 50)
    ax.legend(CLASSES)
    ax.grid()

# procesa las imágenes para recibir el valor codificado de cada una
x_train_encoded = CLUSTmodel.predict(x_train_prep)

# muestra el gráfico con originales con PCA (para que tenga 2 dimensiones)
genera_grafico_pca(x_train, y_train, "Representación de Datos Originales con PCA")

# muestra estadísticas de los datos codificados
minArClust = np.empty(num_features)
minArClust.fill(9999.99)
maxArClust = np.empty(num_features)
maxArClust.fill(-9999.99)
sumArClust = np.zeros(num_features)
for val in x_train_encoded:
  for i in range(num_features):
      sumArClust[i] = sumArClust[i]+val[i]
      if val[i]<minArClust[i]: 
          minArClust[i] = val[i]
      if val[i]>maxArClust[i]: 
          maxArClust[i] = val[i]
print("\n\n> Estadísticas de Clutering de Datos Originales codificado en ", num_features," valores: ")
print("- Mínimos:   ", minArClust)
print("- Máximos:   ", maxArClust)
print("- Totales:   ", sumArClust)
print("- Promedios: ", sumArClust/len(x_train_encoded))
print("\n\n")

# muestra el gráfico codificado (directo si  tiene 2 dimensiones, sino con PCA)
if num_features==2:
    genera_grafico(x_train_encoded, y_train, "Representación de Clustering")
else:
    genera_grafico_pca(x_train_encoded, y_train, "Representación de Datos Originales con PCA")

In [ ]:
print(x_train_prep)

*   Generar y usar el modelo Decoder para generar nuevas datos similares a los entrenadas:

In [ ]:
## Generar el sub-modelo Decoder para Generator
## (desde features hasta output)

# genera una copia del modelo DAE original para evitar romperlo
auxiCloneModel = keras.models.model_from_json(DAEmodel.to_json())
#auxiCloneModel.summary()

# genera la nueva estructura del Generator
input_gen = Input(shape=(num_features,), name='input_gen') # nueva capa de entrada
auxLay_gen = input_gen
for pos in range(len(DAEmodel.layers)):

  # obtiene el nombre de la capa actual
  auxName = DAEmodel.layers[pos].name  
  
  # sólo considera las capas luego de features (decoder y output)
  # para copiar los pesos del DAE original y actualizar la estrcutura
  if auxName.startswith('dec_') or auxName=='output_data':
    auxiCloneModel.layers[pos].set_weights(DAEmodel.layers[pos].get_weights()) 
    auxLay_gen = auxiCloneModel.layers[pos](auxLay_gen) 

# crea el nuevo modelo Generator
GENmodel = Model(input_gen, auxLay_gen, name = 'Decoder/Generator')

print("> Modelo Decoder: ")
GENmodel.summary()
plot_model(GENmodel, show_layer_names=True, show_shapes=True)

In [ ]:
# ejecuta el Generator
#  usando valores definidos al azar como datos de entrada
cantGenerar = 3

# genera los datos de entrada
# (como la codificación tiene varias posiciones con ceros 
# se considera que sólo se ponen al azar entre 10% y el 70% de los valores, el resto queda en cero,
# --sino se podría hacer con " np.random.rand(cantGenerar, num_features) "-- )
arX = []
minRnd = 1
maxRnd = num_features
for i in range(cantGenerar):
  X = np.zeros(num_features)
  for j in range(np.random.randint(low=minRnd, high=maxRnd)):
      pos = np.random.randint(low=0, high=num_features-1)
      X[pos] = np.random.normal()
      
  arX.append( X )

# ejecuta el modelo Generator
reconstr_data_gen = GENmodel.predict( np.array(arX).reshape((len(arX), num_features)) )  
 
# muestra las imágenes generadas
print("\n> Resultados: ")
# muestra las 15 primeras imágenes 
print("\n>Ejemplos de Resultados: ")
for i in range(len(arX)):
      
    # muestra la real
    print("  + Randoms: ", arX[i])
    # muestra la generada por el modelo
    print("  * Constr.: ", reconstr_data_gen[i])
    print("\n")  